3. DATA PREPARATION

In [2]:
import pandas as pd
import numpy as np

loading the data

In [3]:

admissions = pd.read_csv('data/hosp/admissions.csv')
omr = pd.read_csv('data/hosp/omr.csv')
diagnoses = pd.read_csv('data/hosp/diagnoses_icd.csv')
services = pd.read_csv('data/hosp/services.csv')
labevents = pd.read_csv('data/hosp/labevents.csv')
patients=pd.read_csv("data/hosp/patients.csv")
diagnoses2=pd.read_csv("data/hosp/d_icd_diagnoses.csv")
prescriptions=pd.read_csv("data/hosp/prescriptions.csv")

In [4]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaN,URGENT,P03YMR,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0
1,10009628,25926192,2153-09-17 17:08:00,2153-09-25 13:20:00,NaN,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO - PUERTO RICAN,NaN,NaN,0
2,10018081,23983182,2134-08-18 02:02:00,2134-08-23 19:35:00,NaN,URGENT,P233F6,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17 16:24:00,2134-08-18 03:15:00,0
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04 18:46:00,2113-08-06 20:57:00,NaN,URGENT,P07HDB,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0


In [5]:
omr.head()

,subject_id,chartdate,seq_num,result_name,result_value
0,10011398,2146-12-01,1,Height (Inches),63
1,10011398,2147-01-22,1,Weight (Lbs),127
2,10011398,2146-12-01,1,Weight (Lbs),135
3,10011398,2147-07-24,1,Weight (Lbs),136
4,10011398,2147-03-26,1,Weight (Lbs),136


In [6]:
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10014729,F,21,2125,2011 - 2013,NaN
1,10003400,F,72,2134,2011 - 2013,2137-09-02
2,10002428,F,80,2155,2011 - 2013,NaN
3,10032725,F,38,2143,2011 - 2013,2143-03-30
4,10027445,F,48,2142,2011 - 2013,2146-02-09


In [7]:
prescriptions.head()

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10027602,28166872,27168639,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Fentanyl Citrate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10027602,28166872,40720238,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Fentanyl Citrate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10027602,28166872,62845687,NaN,NaN,NaN,2201-10-31 12:00:00,NaN,MAIN,Lorazepam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10027602,28166872,24340150,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Midazolam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10027602,28166872,14435820,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Midazolam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
labevents.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,172061,10014354,29600294.0,1808066,51277,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.40,%,10.5,15.5,NaN,ROUTINE,NaN
1,172062,10014354,29600294.0,1808066,51279,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,3.35,3.35,m/uL,4.6,6.1,abnormal,ROUTINE,NaN
2,172068,10014354,29600294.0,1808066,52172,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.70,fL,35.1,46.3,abnormal,ROUTINE,NaN
3,172063,10014354,29600294.0,1808066,51301,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.30,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
4,172050,10014354,29600294.0,1808066,51249,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.10,g/dL,32.0,37.0,abnormal,ROUTINE,NaN


In [9]:
services.head()

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10001725,25563031,2110-04-11 15:09:36,NaN,GYN
1,10019003,28003918,2148-12-21 03:32:53,NaN,GYN
2,10007818,22987108,2146-06-10 16:38:18,NaN,MED
3,10004235,24181354,2196-02-24 14:39:31,NaN,MED
4,10026255,22059910,2201-07-07 18:16:14,NaN,MED


In [10]:
diagnoses.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10035185,22580999,3,4139,9
1,10035185,22580999,10,V707,9
2,10035185,22580999,1,41401,9
3,10035185,22580999,9,3899,9
4,10035185,22580999,11,V8532,9


In [11]:
diagnoses2.head()

,icd_code,icd_version,long_title
0,0090,9,"Infectious colitis, enteritis, and gastroenter..."
1,01160,9,"Tuberculous pneumonia [any form], unspecified"
2,01186,9,"Other specified pulmonary tuberculosis, tuberc..."
3,01200,9,"Tuberculous pleurisy, unspecified"
4,01236,9,"Tuberculous laryngitis, tubercle bacilli not f..."


In [12]:
merged_diagnoses= pd.merge(diagnoses, diagnoses2, on=['icd_code', 'icd_version'], how='left')
merged_diagnoses.drop(["icd_code", "icd_version","seq_num"],axis=1,inplace=True)
merged_diagnoses.head()


,subject_id,hadm_id,long_title
0,10035185,22580999,Other and unspecified angina pectoris
1,10035185,22580999,Examination of participant in clinical trial
2,10035185,22580999,Coronary atherosclerosis of native coronary ar...
3,10035185,22580999,Unspecified hearing loss
4,10035185,22580999,"Body Mass Index 32.0-32.9, adult"


In [13]:
death_df=admissions[["subject_id","hadm_id","deathtime"]]
death_df["died"]=death_df["deathtime"].isna()==False
death_df.drop("deathtime",axis=1,inplace=True)
death_df.head()

C:\Users\pc\AppData\Local\Temp\ipykernel_308\4074254231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  death_df["died"]=death_df["deathtime"].isna()==False
C:\Users\pc\AppData\Local\Temp\ipykernel_308\4074254231.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  death_df.drop("deathtime",axis=1,inplace=True)


,subject_id,hadm_id,died
0,10004235,24181354,False
1,10009628,25926192,False
2,10018081,23983182,False
3,10006053,22942076,True
4,10031404,21606243,False


In [14]:
#Merge the death df with the merged diagnoses dataframe

merged_df=pd.merge(merged_diagnoses, death_df, on=["subject_id","hadm_id"], how="left")
merged_df["died"].fillna(False, inplace=True)
diagnoses_death_df=merged_df
diagnoses_death_df

,subject_id,hadm_id,long_title,died
0,10035185,22580999,Other and unspecified angina pectoris,False
1,10035185,22580999,Examination of participant in clinical trial,False
2,10035185,22580999,Coronary atherosclerosis of native coronary ar...,False
3,10035185,22580999,Unspecified hearing loss,False
4,10035185,22580999,"Body Mass Index 32.0-32.9, adult",False
...,...,...,...,...
4501,10004733,27411876,Unspecified disturbance of conduct,False
4502,10004733,27411876,"Anxiety state, unspecified",False
4503,10004733,27411876,Unspecified sinusitis (chronic),False
4504,10004733,27411876,Diverticulosis of colon (without mention of he...,False


In [15]:
# merging the filtered_df dataframe with the diagnoses_death_df dataframe using the subject_id column as the key. 
filtered_df=patients[["subject_id","gender","anchor_age"]]
diagnoses_death_df=pd.merge(filtered_df, diagnoses_death_df, on="subject_id", how="right")
#rename anchor_age to age

diagnoses_death_df.rename(columns={"anchor_age":"age"}, inplace=True)
diagnoses_death_df

,subject_id,gender,age,hadm_id,long_title,died
0,10035185,M,70,22580999,Other and unspecified angina pectoris,False
1,10035185,M,70,22580999,Examination of participant in clinical trial,False
2,10035185,M,70,22580999,Coronary atherosclerosis of native coronary ar...,False
3,10035185,M,70,22580999,Unspecified hearing loss,False
4,10035185,M,70,22580999,"Body Mass Index 32.0-32.9, adult",False
...,...,...,...,...,...,...
4501,10004733,M,51,27411876,Unspecified disturbance of conduct,False
4502,10004733,M,51,27411876,"Anxiety state, unspecified",False
4503,10004733,M,51,27411876,Unspecified sinusitis (chronic),False
4504,10004733,M,51,27411876,Diverticulosis of colon (without mention of he...,False


In [16]:
admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaN,URGENT,P03YMR,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0
1,10009628,25926192,2153-09-17 17:08:00,2153-09-25 13:20:00,NaN,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO - PUERTO RICAN,NaN,NaN,0
2,10018081,23983182,2134-08-18 02:02:00,2134-08-23 19:35:00,NaN,URGENT,P233F6,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17 16:24:00,2134-08-18 03:15:00,0
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04 18:46:00,2113-08-06 20:57:00,NaN,URGENT,P07HDB,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,10038992,24745425,2187-07-29 01:05:00,2187-08-03 17:02:00,NaN,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
271,10008287,22168393,2145-09-28 01:17:00,2145-10-02 13:35:00,NaN,SURGICAL SAME DAY ADMISSION,P898NM,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,NaN,NaN,0
272,10022880,27708593,2177-03-12 07:15:00,2177-03-19 14:25:00,NaN,SURGICAL SAME DAY ADMISSION,P99698,PHYSICIAN REFERRAL,HOME,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
273,10004457,23251352,2141-12-17 11:00:00,2141-12-21 15:56:00,NaN,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,SINGLE,OTHER,NaN,NaN,0


In [17]:

length_of_stay_df =diagnoses_death_df.copy()
admissions_filtered_df=admissions[["subject_id","hadm_id","admittime","dischtime","marital_status","race","admission_type","admission_location"]]
length_of_stay_df=pd.merge(admissions_filtered_df, length_of_stay_df, on=["subject_id","hadm_id"],how="right")
#convert admittime to datetime format

length_of_stay_df["admittime"]=pd.to_datetime(length_of_stay_df["admittime"])
length_of_stay_df["dischtime"]=pd.to_datetime(length_of_stay_df["dischtime"])

#calculate the length of stay in days

length_of_stay_df["length_of_stay"]=length_of_stay_df["dischtime"]-length_of_stay_df["admittime"]
length_of_stay_df["length_of_stay"]=length_of_stay_df["length_of_stay"].dt.days

length_of_stay_df



,subject_id,hadm_id,admittime,dischtime,marital_status,race,admission_type,admission_location,gender,age,long_title,died,length_of_stay
0,10035185,22580999,2120-05-12 12:53:00,2120-05-17 16:00:00,MARRIED,WHITE,URGENT,TRANSFER FROM HOSPITAL,M,70,Other and unspecified angina pectoris,False,5
1,10035185,22580999,2120-05-12 12:53:00,2120-05-17 16:00:00,MARRIED,WHITE,URGENT,TRANSFER FROM HOSPITAL,M,70,Examination of participant in clinical trial,False,5
2,10035185,22580999,2120-05-12 12:53:00,2120-05-17 16:00:00,MARRIED,WHITE,URGENT,TRANSFER FROM HOSPITAL,M,70,Coronary atherosclerosis of native coronary ar...,False,5
3,10035185,22580999,2120-05-12 12:53:00,2120-05-17 16:00:00,MARRIED,WHITE,URGENT,TRANSFER FROM HOSPITAL,M,70,Unspecified hearing loss,False,5
4,10035185,22580999,2120-05-12 12:53:00,2120-05-17 16:00:00,MARRIED,WHITE,URGENT,TRANSFER FROM HOSPITAL,M,70,"Body Mass Index 32.0-32.9, adult",False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4501,10004733,27411876,2174-12-04 11:28:00,2174-12-27 14:00:00,SINGLE,UNKNOWN,URGENT,TRANSFER FROM HOSPITAL,M,51,Unspecified disturbance of conduct,False,23
4502,10004733,27411876,2174-12-04 11:28:00,2174-12-27 14:00:00,SINGLE,UNKNOWN,URGENT,TRANSFER FROM HOSPITAL,M,51,"Anxiety state, unspecified",False,23
4503,10004733,27411876,2174-12-04 11:28:00,2174-12-27 14:00:00,SINGLE,UNKNOWN,URGENT,TRANSFER FROM HOSPITAL,M,51,Unspecified sinusitis (chronic),False,23
4504,10004733,27411876,2174-12-04 11:28:00,2174-12-27 14:00:00,SINGLE,UNKNOWN,URGENT,TRANSFER FROM HOSPITAL,M,51,Diverticulosis of colon (without mention of he...,False,23


In [18]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaN,URGENT,P03YMR,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0
1,10009628,25926192,2153-09-17 17:08:00,2153-09-25 13:20:00,NaN,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO - PUERTO RICAN,NaN,NaN,0
2,10018081,23983182,2134-08-18 02:02:00,2134-08-23 19:35:00,NaN,URGENT,P233F6,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17 16:24:00,2134-08-18 03:15:00,0
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04 18:46:00,2113-08-06 20:57:00,NaN,URGENT,P07HDB,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0


In [19]:
readmission_df = length_of_stay_df.copy()

readmission_df = readmission_df.sort_values(["subject_id", "dischtime"])
#find time between every admission and previous discharge for each patient
readmission_df['time_between_admissions'] = readmission_df.groupby('subject_id')['dischtime'].diff()
readmission_df


,subject_id,hadm_id,admittime,dischtime,marital_status,race,admission_type,admission_location,gender,age,long_title,died,length_of_stay,time_between_admissions
2373,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,"Chronic airway obstruction, not elsewhere clas...",False,0,NaT
2374,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Cirrhosis of liver without mention of alcohol,False,0,0 days
2375,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Portal hypertension,False,0,0 days
2376,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Personal history of tobacco use,False,0,0 days
2377,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Other ascites,False,0,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,Enterococcus as the cause of diseases classifi...,False,12,0 days
3783,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,"Diarrhea, unspecified",False,12,0 days
3784,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,Acute posthemorrhagic anemia,False,12,0 days
3785,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,"Surgical operation with anastomosis, bypass or...",False,12,0 days


In [20]:
new_length_of_stay_df=length_of_stay_df.copy()
filtered_prescriptions_df =prescriptions[["subject_id","hadm_id","drug_type","drug"]]
new_length_of_stay_df=pd.merge(filtered_prescriptions_df, new_length_of_stay_df, on=["subject_id","hadm_id"],how="left")
#
new_length_of_stay_df.drop_duplicates(inplace=True)
new_length_of_stay_df


,subject_id,hadm_id,drug_type,drug,admittime,dischtime,marital_status,race,admission_type,admission_location,gender,age,long_title,died,length_of_stay
0,10027602,28166872,MAIN,Fentanyl Citrate,2201-10-30 12:05:00,2201-11-20 14:45:00,SINGLE,WHITE,EW EMER.,EMERGENCY ROOM,F,71,"Urinary tract infection, site not specified",False,21
1,10027602,28166872,MAIN,Fentanyl Citrate,2201-10-30 12:05:00,2201-11-20 14:45:00,SINGLE,WHITE,EW EMER.,EMERGENCY ROOM,F,71,Acute respiratory failure,False,21
2,10027602,28166872,MAIN,Fentanyl Citrate,2201-10-30 12:05:00,2201-11-20 14:45:00,SINGLE,WHITE,EW EMER.,EMERGENCY ROOM,F,71,"Asthma, unspecified type, unspecified",False,21
3,10027602,28166872,MAIN,Fentanyl Citrate,2201-10-30 12:05:00,2201-11-20 14:45:00,SINGLE,WHITE,EW EMER.,EMERGENCY ROOM,F,71,Solitary pulmonary nodule,False,21
4,10027602,28166872,MAIN,Fentanyl Citrate,2201-10-30 12:05:00,2201-11-20 14:45:00,SINGLE,WHITE,EW EMER.,EMERGENCY ROOM,F,71,Streptococcus infection in conditions classifi...,False,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384538,10014354,26228185,MAIN,Carbamide Peroxide 6.5%,2150-04-30 20:19:00,2150-05-07 14:10:00,SINGLE,WHITE,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,M,60,"Constipation, unspecified",False,6
384539,10014354,26228185,MAIN,Carbamide Peroxide 6.5%,2150-04-30 20:19:00,2150-05-07 14:10:00,SINGLE,WHITE,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,M,60,"Hematuria, unspecified",False,6
384540,10014354,26228185,MAIN,Carbamide Peroxide 6.5%,2150-04-30 20:19:00,2150-05-07 14:10:00,SINGLE,WHITE,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,M,60,Obstructive sleep apnea (adult) (pediatric),False,6
384541,10014354,26228185,MAIN,Carbamide Peroxide 6.5%,2150-04-30 20:19:00,2150-05-07 14:10:00,SINGLE,WHITE,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,M,60,Long term (current) use of anticoagulants,False,6


In [21]:
df=admissions.copy()
df.admittime = pd.to_datetime(df.admittime)   
df.dischtime = pd.to_datetime(df.dischtime)

df1 = df.groupby([df.admittime.dt.year, df.admittime.dt.month]).count().rename_axis(['year', 'month'])['admittime'].reset_index(name='Entries')

df1

,year,month,Entries
0,2110,4,1
1,2110,11,1
2,2111,1,1
3,2111,11,1
4,2112,9,1
...,...,...,...
217,2201,2,1
218,2201,3,1
219,2201,7,1
220,2201,10,1


In [22]:
admissions.duplicated().value_counts()

False    275
Name: count, dtype: int64

In [23]:
readmission_df

,subject_id,hadm_id,admittime,dischtime,marital_status,race,admission_type,admission_location,gender,age,long_title,died,length_of_stay,time_between_admissions
2373,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,"Chronic airway obstruction, not elsewhere clas...",False,0,NaT
2374,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Cirrhosis of liver without mention of alcohol,False,0,0 days
2375,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Portal hypertension,False,0,0 days
2376,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Personal history of tobacco use,False,0,0 days
2377,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Other ascites,False,0,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,Enterococcus as the cause of diseases classifi...,False,12,0 days
3783,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,"Diarrhea, unspecified",False,12,0 days
3784,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,Acute posthemorrhagic anemia,False,12,0 days
3785,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,"Surgical operation with anastomosis, bypass or...",False,12,0 days


In [27]:

readmission_df['admittime'] = pd.to_datetime(readmission_df['admittime'])
readmission_df['dischtime'] = pd.to_datetime(readmission_df['dischtime'])

# Sort by subject_id and admittime
readmission_df = readmission_df.sort_values(by=['subject_id', 'admittime'])

# Calculate the time difference between each discharge and the next admission
readmission_df['next_admittime'] = readmission_df.groupby('subject_id')['admittime'].shift(-1)
readmission_df['readmission_time'] = readmission_df['next_admittime'] - readmission_df['dischtime']

# Define a readmission threshold (e.g., 30 days)
readmission_threshold = pd.Timedelta(days=30)

# Flag readmissions based on the threshold
readmission_df['is_readmission'] = (readmission_df['readmission_time'] <= readmission_threshold) & (readmission_df['readmission_time'] > pd.Timedelta(days=0))

# Count the number of readmissions for each subject
readmission_counts = readmission_df.groupby('subject_id')['is_readmission'].sum()

# Map the readmission counts back to the original DataFrame
readmission_df['readmissions'] = readmission_df['subject_id'].map(readmission_counts)
readmission_df





,subject_id,hadm_id,admittime,dischtime,marital_status,race,admission_type,admission_location,gender,age,long_title,died,length_of_stay,time_between_admissions,number_of_readmissions,next_admittime,readmission_time,is_readmission,readmissions
2373,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,"Chronic airway obstruction, not elsewhere clas...",False,0,NaT,39,2180-05-06 22:23:00,-1 days +05:08:00,False,2
2374,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Cirrhosis of liver without mention of alcohol,False,0,0 days,39,2180-05-06 22:23:00,-1 days +05:08:00,False,2
2375,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Portal hypertension,False,0,0 days,39,2180-05-06 22:23:00,-1 days +05:08:00,False,2
2376,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Personal history of tobacco use,False,0,0 days,39,2180-05-06 22:23:00,-1 days +05:08:00,False,2
2377,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,WIDOWED,WHITE,URGENT,TRANSFER FROM HOSPITAL,F,52,Other ascites,False,0,0 days,39,2180-05-06 22:23:00,-1 days +05:08:00,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,Enterococcus as the cause of diseases classifi...,False,12,0 days,259,2148-01-23 12:18:00,-13 days +15:27:00,False,4
3783,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,"Diarrhea, unspecified",False,12,0 days,259,2148-01-23 12:18:00,-13 days +15:27:00,False,4
3784,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,Acute posthemorrhagic anemia,False,12,0 days,259,2148-01-23 12:18:00,-13 days +15:27:00,False,4
3785,10040025,27996267,2148-01-23 12:18:00,2148-02-04 20:51:00,DIVORCED,WHITE,OBSERVATION ADMIT,TRANSFER FROM SKILLED NURSING FACILITY,F,64,"Surgical operation with anastomosis, bypass or...",False,12,0 days,259,2148-01-23 12:18:00,-13 days +15:27:00,False,4
